In [ ]:
import os
import shutil
import flopy
# import specific FloPy modules
from flopy.utils.gridgen import Gridgen 
from flopy.utils.gridintersect import GridIntersect

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

# for working with shapefiles
import shapefile as sf
from shapely.geometry import Polygon, LineString, MultiLineString, Point

print(f'Flopy version:{flopy.__version__}')

In [ ]:
# define the location of the model working directory; this is where all the MF6 files will be written
# the folder path is relative to the location in which python is running. In our case, relative to the location of the jupyter notebok file.

# set the path to the history model
hist_workspace = os.path.join('..','models','sgn_model_1layer_transport')

# set the new workspace
workspace = os.path.join('..','models','sgn_model_1layer_scenario')

# make sure that the folder exists. If it doesn't, create it.
# copy the hisotry model to the new folder
if os.path.exists(workspace):
    shutil.rmtree(workspace)
shutil.copytree(hist_workspace, workspace)    
    
# the relative pathto the input file folder
datadir = os.path.join('..','data','sgn')

# the relative path to the executables folder
bindir = os.path.join('..','bin')

In [ ]:
# load the model
sim = flopy.mf6.MFSimulation.load(sim_ws=workspace, verbosity_level=1)

print(sim.model_names)

In [ ]:
gwf = sim.get_model("sgn")

In [ ]:
# GridIntersect works by first calling an intersect object on the model grid:
ix = GridIntersect(gwf.modelgrid)

In [ ]:
# hydraulic barrier data
barrier_xy = Point(1519614., 5031870.)
print(barrier_xy)

inj_rate = 30/1000 #30 l/s to m3/s

In [ ]:
# ineresect all model cells
barrier_cell = ix.intersect(barrier_xy).cellids

In [ ]:
gwf.wel.stress_period_data

In [ ]:
# append the barrier wells to the wel package SPD
for i in barrier_cell:
    gwf.wel.stress_period_data.append_data([((0, i), inj_rate, 'hydraulic_barrier')])

In [ ]:
# check if that worked; see the end of the array
gwf.wel.stress_period_data

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()

In [ ]:
fname = os.path.join(workspace, gwf.name + '.hds')
hdobj = flopy.utils.HeadFile(fname, model=gwf)
head = hdobj.get_alldata()

hmin = head.min()
hmax = head.max()

print(f'Max head in model run: {hmax}')
print(f'Min head in model run: {hmin}')

head.shape

In [ ]:
# get the times at which values are recorede in the heads file
hdtimes = hdobj.get_times()
# generic plot of head data
hdobj.plot(mflay=0, totim=hdtimes[0] ,colorbar=True, contour=True, clabel=True)

In [ ]:
# Visualy compare effect of inkection well

#read the .hds file form the hisotry model
fname2 = os.path.join(hist_workspace, gwf.name + '.hds')
hist_hdobj = flopy.utils.HeadFile(fname2, model=gwf)
hist_hdobj.plot(mflay=0, totim=hdtimes[0] ,colorbar=True, contour=True, clabel=True)

In [ ]:
hdobj.get_kstpkper()

In [ ]:
# read the grd file
fname = os.path.join(workspace, f'{gwf.name}.disv.grb')
grd = flopy.utils.MfGrdFile(fname, verbose=False)
iverts, verts = grd.get_verts()
vertc = grd.get_centroids()
mg = grd.get_modelgrid()

deltah = hdobj.get_data(mflay=0, kstpkper=(3,0)) - hist_hdobj.get_data(mflay=0,  kstpkper=(3,0))

In [ ]:
fig = plt.figure(figsize=(5, 5), tight_layout=True)
ax = fig.add_subplot(111, aspect='equal')

ext_offset = 500
extent=(1519614-ext_offset, 1519614+ext_offset,
        5031870-ext_offset, 5031870+ext_offset)

mm = flopy.plot.PlotMapView(model=gwf, ax=ax, layer=0, extent=extent) 

# plot the model grid + fill contours
dh = mm.plot_array(edgecolor='grey', a=deltah[0], cmap='Blues', alpha=1, linewidth=0.1)
cb = plt.colorbar(dh, shrink=0.5, pad=0.01)

# plot the site shp
flopy.plot.plot_shapefile(sf.Reader(os.path.join(datadir, 'shp','Site_area.shp')), linewidth=2, facecolor='grey', edgecolor='w', alpha=0.3);

In [ ]:
 #Get the MF6 concentration output
fname_mf6 = os.path.join(workspace, list(sim.model_names)[1] + ".ucn")
ucnobj_mf6 = flopy.utils.HeadFile(fname_mf6, precision="double", text="CONCENTRATION")

# pre-hydraulic barrier simulation
fname_mf6 = os.path.join(hist_workspace, list(sim.model_names)[1] + ".ucn")
hist_ucnobj_mf6 = flopy.utils.HeadFile(fname_mf6, precision="double", text="CONCENTRATION")

# difference between them
deltac = ucnobj_mf6.get_data(mflay=0, kstpkper=(3,0)) - hist_ucnobj_mf6.get_data(mflay=0,  kstpkper=(3,0))

In [ ]:
# differnec ein concentration

fig = plt.figure(figsize=(5, 5), tight_layout=True)
ax = fig.add_subplot(111, aspect='equal')

extent=(1519100, 1520300, #xmin, xmax
        5031600, 5032600) #ymin, ymax

mm = flopy.plot.PlotMapView(model=gwf, ax=ax, layer=0, extent=extent) 

# plot the differnec ein concnetration induced by the hydraulic barrier
a = deltac[0].copy()
a[(a<0.01) & (a>-0.01)] = np.nan

dc = mm.plot_array(edgecolor='grey', a=a, cmap='RdYlBu_r', alpha=1, linewidth=0.1)
cb = plt.colorbar(dc, shrink=0.5, pad=0.01)

# plot the site shp
flopy.plot.plot_shapefile(sf.Reader(os.path.join(datadir, 'shp','Site_area.shp')), linewidth=2, facecolor='grey', edgecolor='w', alpha=0.3);

# plot the well cells
wells = mm.plot_bc("WEL")